In [22]:
import pandas as pd
import numpy as np
#from get_data_from_nbaApi import get_player_bio
#from Scrape_data_from_ESPN import get_players_PER_stats, url_params

The data was prepared using the main.py script, which calls:

get_data_from_nbaApi.py - Raw data from NBA API. <br>
Scrape_data_from_ESPN.py - Scraped data from ESPN website. <br>
data_handle.py - Combine the scripts, adds calculations, continent and more. <br>

The data was pulled starting from 2003 season <br>
Let's have a look at the data

In [84]:
df = pd.read_csv('final_df.csv')
print(df.head())
print(df.describe())
print(df.isna().sum())

      PLAYER_NAME  year    PER COUNTRY      continent   AGE DRAFT_YEAR  \
0   Kevin Garnett  2003  26.46     USA  North America  27.0       1995   
1   Desmond Mason  2003  15.40     USA  North America  25.0       2000   
2     Troy Murphy  2003  15.38     USA  North America  23.0       2001   
3  Scottie Pippen  2003  15.36     USA  North America  37.0       1987   
4     Vlade Divac  2003  15.25  Serbia         Europe  35.0       1989   

   year_max_per  max_per  player_min_year  
0          2004    29.50             2003  
1          2004    15.66             2003  
2          2010    18.07             2003  
3          2003    15.36             2003  
4          2004    16.29             2003  
              year          PER          AGE  year_max_per      max_per  \
count  6863.000000  6863.000000  6807.000000   6863.000000  6863.000000   
mean   2012.770363    14.331237    27.045541   2012.300160    17.385353   
std       5.766744     4.328504     4.203707      6.067814     4.6

First, we see some nulls to take care. <br>
We need to decide which years are relevant.

In [34]:
piv = df.pivot_table(index = 'DRAFT_YEAR',
                     columns='year',
                     values='PER',
                     aggfunc='count',
                     fill_value = 0)

print(piv)


year        2003  2004  2005  2006  2007  2008  2009  2010  2011  2012  2013  \
DRAFT_YEAR                                                                     
1984           3     0     0     0     0     0     0     0     0     0     0   
1985           2     1     0     0     0     0     0     0     0     0     0   
1986           1     0     0     0     0     0     0     0     0     0     0   
1987           4     3     1     0     0     0     0     0     0     0     0   
1988           4     1     0     0     0     0     0     0     0     0     0   
1989           5     2     1     1     1     0     0     0     0     0     0   
1990          10     7     4     3     1     0     0     0     0     0     0   
1991           5     6     4     1     1     1     0     0     0     0     0   
1992          12    12    11     5     4     2     1     1     1     0     0   
1993          17    16    12     3     3     1     0     0     0     0     0   
1994          14    14    14     7     5

In [86]:
df_undrafted = df[df['DRAFT_YEAR']=='Undrafted'].reset_index()
print(df_undrafted.nlargest(5,'PER'))


undrafted_pivot = df_undrafted.pivot_table(
    df_undrafted[['PER']],
    index = 'year',
    aggfunc= ('count','max','mean')
)

print(undrafted_pivot)

     index       PLAYER_NAME  year    PER      COUNTRY      continent   AGE  \
480   4454  Boban Marjanovic  2016  27.77       Serbia         Europe  27.0   
603   5526  Boban Marjanovic  2019  24.25       Serbia         Europe  30.0   
686   6033    Christian Wood  2020  23.22          USA  North America  24.0   
702   6212     Chris Boucher  2021  21.97  Saint Lucia  North America  28.0   
182   1741     Jose Calderon  2008  20.77        Spain         Europe  26.0   

    DRAFT_YEAR  year_max_per  max_per  player_min_year  
480  Undrafted          2016    27.77             2016  
603  Undrafted          2016    27.77             2016  
686  Undrafted          2020    23.22             2020  
702  Undrafted          2021    21.97             2020  
182  Undrafted          2008    20.77             2006  
       PER                  
     count    max       mean
year                        
2003    33  18.80  11.116970
2004    32  20.13  12.499375
2005    33  20.72  12.464242
2006    3

Looks like there is some great information we're missing. We'll get back to this later. <br>
For now, let's check the hypothesis on the drafted players only (For simplicity). <br>

Clean data - nulls, use the relevant years (So the players have time to reach the maximum PER). <br>
Calculate the KPI- Time to reach the maximum PER (in Years). <br>
Defined as the difference between the year when the player reached the maximum PER and the Draft year.

In [88]:
df_use = df[df['DRAFT_YEAR']!='Undrafted']
df_use = df_use[df_use['DRAFT_YEAR'].notna()]
print(df_use.isna().sum())

df_use = df_use[(df_use['DRAFT_YEAR'].astype(int) < 2017) & (df_use['DRAFT_YEAR'].astype(int) > 2003)]

df_use['time_to_max'] = df_use['year_max_per'] - df_use['DRAFT_YEAR'].astype(int)

print(df_use.describe())
print(df_use.head())

PLAYER_NAME        0
year               0
PER                0
COUNTRY            0
continent          0
AGE                0
DRAFT_YEAR         0
year_max_per       0
max_per            0
player_min_year    0
dtype: int64
              year          PER          AGE  year_max_per      max_per  \
count  3343.000000  3343.000000  3343.000000   3343.000000  3343.000000   
mean   2014.910260    14.894921    25.918935   2014.906372    18.109698   
std       4.246838     4.303676     3.515250      4.153535     4.535793   
min    2005.000000     0.000000    19.000000   2005.000000     0.000000   
25%    2012.000000    11.935000    23.000000   2012.000000    15.090000   
50%    2015.000000    14.480000    26.000000   2015.000000    17.670000   
75%    2018.000000    17.345000    28.000000   2018.000000    20.810000   
max    2022.000000    32.940000    38.000000   2022.000000    32.940000   

       player_min_year  time_to_max  
count      3343.000000  3343.000000  
mean       2010.776548   

The years taken: <br>
Drafted before 2017 (let's give all players at least 5 years of Per). <br>
Drafted after 2003 (Start from players with 0 experience). <br>

Now, Let's check the hypothesis and see the time that takes to the players to reach the maximum PER, by continent.

In [82]:
pivot_by_cont = df_use.pivot_table(
     df_use[['time_to_max']],
     index = 'continent'
)
print(pivot_by_cont)

               time_to_max
continent                 
Africa            5.921053
Asia              6.584906
Europe            6.322816
North America     5.404482
Oceania           5.800000
South America     6.233333
